# One run full walktrhough 

* Do the full walk through on the large data set
* Refactor the source code and bring it to individual scripts
* Ensure a full run with one click

In [26]:
## check some parameters
## depending where you launch your notebook, the relative path might not work
## you should start the notebook server from your base path
## when opening the notebook, typically your path will be ../ads_covid-19/notebooks
import os
if os.path.split(os.getcwd())[-1]=='notebooks':
    os.chdir("../")

'Your base path is at: '+os.path.split(os.getcwd())[-1]

'Your base path is at: code_Covid_19_data_analysis'

## 1 Update all data

### GDP prompt  clean it

#### task 1: Clean up code

Clean up code and document the following code. Use pep-8 guidelines and the possibility to use doxygen for automated code documentation. ''' ...code ...'''

#### task 2: ask for explanation: 

what is this command: subprocess.Popen


In [28]:
# %load src/data/get_data.py

import subprocess
import os

import pandas as pd
import numpy as np

from datetime import datetime

import requests
import json

def get_johns_hopkins():
    """
    Retrieve data using a git pull request.
    
    The source code must be pulled first. The result is stored in the predefined CSV structure.
    
    :return: None
    """
    git_pull_command = "/usr/bin/git pull"
    covid_data_directory = "../COVID-19/"  # take care for the correct path
    
    git_pull = subprocess.Popen(
        git_pull_command,
        cwd=os.path.dirname(covid_data_directory),
        shell=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    
    out, error = git_pull.communicate()

    if error:
        print(f"Error: {error.decode('utf-8')}")
    
    if out:
        print(f"Output: {out.decode('utf-8')}")


if __name__ == '__main__':
    get_johns_hopkins()



Output: Already up to date.



## 2. Process pipeline 

### GDP prompt  clean it

#### task 3: Simplify code and document it 

please write this code in a simpler form and explain the changes.
Try to use less commands and clearer documentation

''' ... your code ...'''

#### output for changes

changes made:

* Shortened the function name to store_relational_jh_data to follow the PEP-8 guidelines for lowercase function names with words separated by underscores.
* Combined the column renaming and filling missing state values into one line using the fillna function with a dictionary.
* Replaced the chained operations for transforming the dataset into a relational model with the melt function, which is more concise and easier to understand.
* Used f-strings for printing the output, which makes the code cleaner and more readable.
* Added comments to explain each step of the code.

In [29]:
# %load src/data/process_JH_data.py
import pandas as pd
import numpy as np

from datetime import datetime


def store_relational_JH_data():
    """
    Transform the COVID-19 data into a relational dataset and save it as a CSV file.
    """

    # Read the raw data from the CSV file
    data_path = '../COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
    data = pd.read_csv(data_path)

    # Rename columns and fill missing state values with 'no'
    data = data.rename(columns={'Country/Region': 'country', 'Province/State': 'state'}).fillna({'state': 'no'})

    # Drop unnecessary columns
    data = data.drop(['Lat', 'Long'], axis=1)

    # Transform the dataset into a relational model
    relational_data = data.melt(id_vars=['state', 'country'], var_name='date', value_name='confirmed')
    relational_data['date'] = pd.to_datetime(relational_data['date'])

    # Save the relational model to a CSV file
    output_file = 'data/processed/COVID_relational_confirmed.csv'
    relational_data.to_csv(output_file, sep=';', index=False)

    print(f'Number of rows stored: {relational_data.shape[0]}')
    print(f'Latest date is: {relational_data["date"].max()}')
    
    
if __name__ == '__main__':

    store_relational_JH_data()


Number of rows stored: 330327
Latest date is: 2023-03-09 00:00:00


# Strip down to a smaller file

attention we are overwriting the existing file

In [30]:
#save the smaller file
countries_to_extract = ['US','Germany', 'India','France'] # add your country of interest

pd_result_larg =pd.read_csv('data/processed/COVID_relational_confirmed.csv', sep=';', parse_dates=[0])
pd_result_larg = pd_result_larg[pd_result_larg['country'].isin(countries_to_extract)]

# Save the relational model to a CSV file
output_file = 'data/processed/COVID_relational_confirmed.csv'
pd_result_larg.to_csv(output_file, sep=';', index=False)

## 3  Filter and Doubling Rate Calculation




#### GDP prompt  to accelerate the function

####  task 4: help for profiling

How can I profile the code. my goal is to understand where I need most of the time. Each function should be listet how long it took  

####  task 5: use an other library (even a wrong librarary stated)

the code is not faster use arrow to accelarte it


####  task 6: iterative search for error

this code does not work, where is the error. the error message is below as well ... copy error and code ...



In [33]:
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd
import time

import dask.dataframe as dd

from scipy import signal


def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope


def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function (data structure kept)

        parameters:
        ----------
        df_input : pandas.series
        column : str
        window : int
            used data points to calculate the filter result

        Returns:
        ----------
        df_result: pd.DataFrame
            the index of the df_input has to be preserved in result
    '''

    degree=1
    df_result=df_input

    filter_in=df_input[column].fillna(0) # attention with the neutral element here

    result=signal.savgol_filter(np.array(filter_in),
                           window, # window size used for filtering
                           1)
    df_result[str(column+'_filtered')]=result
    return df_result

def rolling_reg(df_input,col='confirmed'):
    ''' Rolling Regression to approximate the doubling time'

        Parameters:
        ----------
        df_input: pd.DataFrame
        col: str
            defines the used column
        Returns:
        ----------
        result: pd.DataFrame
    '''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)



    return result




def calc_filtered_data(df_input,filter_on='confirmed'):
    '''  Calculate savgol filter and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'

    df_output=df_input.copy() # we need a copy here otherwise the filter_on column will be overwritten

    pd_filtered_result=df_output[['state','country',filter_on]].groupby(['state','country']).apply(savgol_filter)#.reset_index()

    #print('--+++ after group by apply')
    #print(pd_filtered_result[pd_filtered_result['country']=='Germany'].tail())

    #df_output=pd.merge(df_output,pd_filtered_result[['index',str(filter_on+'_filtered')]],on=['index'],how='left')
    df_output=pd.merge(df_output,pd_filtered_result[[str(filter_on+'_filtered')]],left_index=True,right_index=True,how='left')
    #print(df_output[df_output['country']=='Germany'].tail())
    return df_output.copy()


    
def calc_doubling_rate(df_input, filter_on='confirmed'):
    """Calculate approximated doubling rate and return merged data frame."""

    # Convert the input DataFrame to a Dask DataFrame
    dask_input = dd.from_pandas(df_input, npartitions=8)  # You can adjust the number of partitions based on your system's resources.

    # Define a custom function for Dask's apply method
    def calculate_doubling_rate(df):
        df[filter_on + '_DR'] = df[filter_on].rolling(window=3, min_periods=3).apply(get_doubling_time_via_regression, raw=False)
        return df

    # Calculate the doubling rate for each group
    meta = df_input.copy()
    meta[filter_on + '_DR'] = np.nan
    dask_result = dask_input.groupby(['state', 'country']).apply(calculate_doubling_rate, meta=meta)

    # Convert the result back to a pandas DataFrame
    df_output = dask_result.compute()

    return df_output

if __name__ == '__main__':
    start_time = time.time()
    test_data_reg = np.array([2, 4, 6])
    result = get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: ' + str(result))
    print(f"Time taken by 'get_doubling_time_via_regression': {time.time() - start_time:.2f} seconds")
    
    start_time = time.time()
    pd_JH_data = pd.read_csv('data/processed/COVID_relational_confirmed.csv', sep=';', parse_dates=[0])
    pd_JH_data = pd_JH_data.sort_values('date', ascending=True).copy()
    print(f"Time taken to read and sort data: {time.time() - start_time:.2f} seconds")

    start_time = time.time()
    pd_result_larg = calc_filtered_data(pd_JH_data)
    print(f"Time taken by 'calc_filtered_data': {time.time() - start_time:.2f} seconds")

    start_time = time.time()
    pd_result_larg = calc_doubling_rate(pd_result_larg)
    print(f"Time taken by 'calc_doubling_rate': {time.time() - start_time:.2f} seconds")

    start_time = time.time()
    pd_result_larg = calc_doubling_rate(pd_result_larg, 'confirmed_filtered')
    print(f"Time taken by 'calc_doubling_rate' with 'confirmed_filtered': {time.time() - start_time:.2f} seconds")

    start_time = time.time()
    mask = pd_result_larg['confirmed'] > 100
    pd_result_larg['confirmed_filtered_DR'] = pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)
    pd_result_larg.to_csv('data/processed/COVID_final_set.csv', sep=';', index=False)
    print(f"Time taken to process and save the final dataset: {time.time() - start_time:.2f} seconds")

    print(pd_result_larg[pd_result_larg['country'] == 'Germany'].tail())

the test slope is: [2.]
Time taken by 'get_doubling_time_via_regression': 0.00 seconds
Time taken to read and sort data: 0.03 seconds
Time taken by 'calc_filtered_data': 0.04 seconds
Time taken by 'calc_doubling_rate': 7.20 seconds
Time taken by 'calc_doubling_rate' with 'confirmed_filtered': 7.16 seconds
Time taken to process and save the final dataset: 0.09 seconds
      state  country        date  confirmed  confirmed_filtered  confirmed_DR  \
10647    no  Germany  2021-12-31    7109182           7100628.0    331.771117   
10662    no  Germany  2022-01-01    7109182           7109182.0           inf   
10677    no  Germany  2022-01-02    7109182           7135027.2           inf   
10692    no  Germany  2022-01-03    7109182           7172654.8           inf   
10707    no  Germany  2022-01-04    7238408           7223150.4    110.693782   

       confirmed_filtered_DR  
10647             379.348284  
10662             565.200582  
10677             413.669256  
10692             2

# 3b  Test function

#### GDP prompt  to accelerate the function

##### task 7: write a test function

please write a test function for the following function to check for correct results
''' code ''''

##### output: 
attention the result was not correct, always cross check correct results



In [43]:
import numpy as np
from sklearn.linear_model import LinearRegression

def test_get_doubling_time_via_regression():
    reg = LinearRegression(fit_intercept=True)

    # Test case 1: Doubling rate = 2
    test_data_1 = np.array([1, 2, 4])
    expected_result_1 = 1.55555556
    result_1 = get_doubling_time_via_regression(test_data_1)
    assert np.isclose(result_1, expected_result_1, rtol=1e-05), f"Expected {expected_result_1}, but got {result_1}"

    # Test case 2: Doubling rate = 3
    test_data_2 = np.array([4, 8, 16])
    expected_result_2 = 1.55555556
    result_2 = get_doubling_time_via_regression(test_data_2)
    assert np.isclose(result_2, expected_result_2, rtol=1e-05), f"Expected {expected_result_2}, but got {result_2}"

    # Test case 3: Doubling rate = 1
    test_data_3 = np.array([1, 2, 4])
    expected_result_3 = 1.55555556
    result_3 = get_doubling_time_via_regression(test_data_3)
    assert np.isclose(result_3, expected_result_3, rtol=1e-05), f"Expected {expected_result_3}, but got {result_3}"

    print("All tests passed!")

test_get_doubling_time_via_regression()

All tests passed!


In [34]:
print(pd_result_larg[pd_result_larg['country']=='US'].tail())

      state country        date  confirmed  confirmed_filtered  confirmed_DR  \
10649    no      US  2021-12-31   54907717          54719451.0     97.959334   
10664    no      US  2022-01-01   55099948          55247702.6    155.624410   
10679    no      US  2022-01-02   55396191          55822126.6    225.742286   
10694    no      US  2022-01-03   56438983          56429298.4     83.112153   
10709    no      US  2022-01-04   57267794          57166945.2     60.234629   

       confirmed_filtered_DR  
10649             126.403018  
10664             115.255426  
10679             100.234545  
10694              94.504470  
10709              83.985736  


## 4 Visual Board

In [35]:
# %load src/visualization/visualize.py
import pandas as pd
import numpy as np

import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State

import plotly.graph_objects as go

import os
print(os.getcwd())
df_input_large=pd.read_csv('data/processed/COVID_final_set.csv',sep=';')


fig = go.Figure()

app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    #  Applied Data Science on COVID-19 data

    Goal of the project is to teach data science by applying a cross industry standard process,
    it covers the full walkthrough of: automated data gathering, data transformations,
    filtering and machine learning to approximating the doubling time, and
    (static) deployment of responsive dashboard.

    '''),

    dcc.Markdown('''
    ## Multi-Select Country for visualization
    '''),


    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in df_input_large['country'].unique()],
        value=['US', 'Germany','Italy'], # which are pre-selected
        multi=True
    ),

    dcc.Markdown('''
        ## Select Timeline of confirmed COVID-19 cases or the approximated doubling time
        '''),


    dcc.Dropdown(
    id='doubling_time',
    options=[
        {'label': 'Timeline Confirmed ', 'value': 'confirmed'},
        {'label': 'Timeline Confirmed Filtered', 'value': 'confirmed_filtered'},
        {'label': 'Timeline Doubling Rate', 'value': 'confirmed_DR'},
        {'label': 'Timeline Doubling Rate Filtered', 'value': 'confirmed_filtered_DR'},
    ],
    value='confirmed',
    multi=False
    ),

    dcc.Graph(figure=fig, id='main_window_slope')
])



@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
    Input('doubling_time', 'value')])
def update_figure(country_list,show_doubling):


    if 'doubling_rate' in show_doubling:
        my_yaxis={'type':"log",
               'title':'Approximated doubling rate over 3 days (larger numbers are better #stayathome)'
              }
    else:
        my_yaxis={'type':"log",
                  'title':'Confirmed infected people (source johns hopkins csse, log-scale)'
              }


    traces = []
    for each in country_list:

        df_plot=df_input_large[df_input_large['country']==each]

        if show_doubling=='doubling_rate_filtered':
            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.mean).reset_index()
        else:
            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.sum).reset_index()
       #print(show_doubling)


        traces.append(dict(x=df_plot.date,
                                y=df_plot[show_doubling],
                                mode='markers+lines',
                                opacity=0.9,
                                name=each
                        )
                )

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        )
    }

if __name__ == '__main__':

    app.run_server(debug=True, use_reloader=False)


/Users/kienlef/Documents/Vorlesung/Covid19_update_2023/code_Covid_19_data_analysis
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
